**Importing and Initializing**

In [1]:
import pyspark

In [107]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, col, when, desc, max, asc, avg, round

In [3]:
spark = SparkSession.builder.appName("Disney Data Analysis").getOrCreate()

**Displaying the Schema**

In [184]:
df.printSchema()

root
 |-- imdb_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- plot: string (nullable = true)
 |-- type: string (nullable = true)
 |-- rated: string (nullable = true)
 |-- year: string (nullable = true)
 |-- released_at: string (nullable = true)
 |-- added_at: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- director: string (nullable = true)
 |-- writer: string (nullable = true)
 |-- actors: string (nullable = true)
 |-- language: string (nullable = true)
 |-- country: string (nullable = true)
 |-- awards: string (nullable = true)
 |-- metascore: string (nullable = true)
 |-- imdb_rating: string (nullable = true)
 |-- imdb_votes: string (nullable = true)



**Reading the CSV File from the local system**

In [4]:
df = spark.read.csv("disney_plus_shows.csv", header = True, inferSchema = True)

**Data Exploration**

**Total number of Columns**

In [6]:
len(df.columns)

19

**Total number of Rows**

In [7]:
df.count()

992

**Checking count of missing values**

In [8]:
df.select([count(when(col(c).isNull(), 1)) for c in df.columns]).show()

+---------------------------------------------+-------------------------------------------+------------------------------------------+------------------------------------------+-------------------------------------------+------------------------------------------+-------------------------------------------------+----------------------------------------------+---------------------------------------------+-------------------------------------------+----------------------------------------------+--------------------------------------------+--------------------------------------------+----------------------------------------------+---------------------------------------------+--------------------------------------------+-----------------------------------------------+-------------------------------------------------+------------------------------------------------+
|count(CASE WHEN (imdb_id IS NULL) THEN 1 END)|count(CASE WHEN (title IS NULL) THEN 1 END)|count(CASE WHEN (plot IS NULL) THEN

In [9]:
df.describe().show()

+-------+---------+---------------+--------------------+-----------------+--------------------+--------------------+------------------+------------------+------------------+--------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|summary|  imdb_id|          title|                plot|             type|               rated|                year|       released_at|          added_at|           runtime|         genre|director|              writer|              actors|            language|             country|              awards|           metascore|         imdb_rating|          imdb_votes|
+-------+---------+---------------+--------------------+-----------------+--------------------+--------------------+------------------+------------------+------------------+--------------+--------+--------------------+--------------------+--------------------+--------

**Column Names**

In [10]:
df.columns

['imdb_id',
 'title',
 'plot',
 'type',
 'rated',
 'year',
 'released_at',
 'added_at',
 'runtime',
 'genre',
 'director',
 'writer',
 'actors',
 'language',
 'country',
 'awards',
 'metascore',
 'imdb_rating',
 'imdb_votes']

**Movie Rating Analysis**

**Selecting required columns and creating a new dataframe for analysis**

In [11]:
df1 = df.select(col("imdb_rating"), col("title"), col("rated"), col("imdb_votes"))
df1.show(10)

+-----------+--------------------+-----+----------+
|imdb_rating|               title|rated|imdb_votes|
+-----------+--------------------+-----+----------+
|        7.3|10 Things I Hate ...|PG-13|   283,945|
|        6.2|101 Dalmatian Street|  N/A|       124|
|        5.7|      101 Dalmatians|    G|    97,785|
|        5.8|101 Dalmatians 2:...|    G|     7,434|
|        4.9|      102 Dalmatians|    G|    33,444|
|        6.3|12 Dates of Chris...|   PG|     4,933|
|        7.2|20,000 Leagues Un...|    G|    27,761|
|         77|        A Bug's Life|movie|       7.2|
|        7.6|A Celebration of ...|  N/A|        35|
|        6.8|       A Goofy Movie|    G|    43,655|
+-----------+--------------------+-----+----------+
only showing top 10 rows



In [22]:
df.select(df["title"], df["imdb_rating"]).show(10, truncate = False)

+------------------------------------------+-----------+
|title                                     |imdb_rating|
+------------------------------------------+-----------+
|10 Things I Hate About You                |7.3        |
|101 Dalmatian Street                      |6.2        |
|101 Dalmatians                            |5.7        |
|101 Dalmatians 2: Patch's London Adventure|5.8        |
|102 Dalmatians                            |4.9        |
|12 Dates of Christmas                     |6.3        |
|20,000 Leagues Under the Sea              |7.2        |
|A Bug's Life                              |77         |
|A Celebration of the Music from Coco      |7.6        |
|A Goofy Movie                             |6.8        |
+------------------------------------------+-----------+
only showing top 10 rows



**Displaying the first three columns**

In [24]:
df.select(df.columns[:3]).show(10)

+----------+--------------------+--------------------+
|   imdb_id|               title|                plot|
+----------+--------------------+--------------------+
| tt0147800|10 Things I Hate ...|A pretty, popular...|
| tt7019028|101 Dalmatian Street|This series follo...|
| tt0115433|      101 Dalmatians|An evil high-fash...|
| tt0324941|101 Dalmatians 2:...|Being one of 101 ...|
| tt0211181|      102 Dalmatians|Cruella DeVil get...|
| tt1846442|12 Dates of Chris...|A story that foll...|
| tt0046672|20,000 Leagues Un...|A ship sent to in...|
| tt0120623|        A Bug's Life|"A misfit ant, lo...|
|tt12076020|A Celebration of ...|                 N/A|
| tt0113198|       A Goofy Movie|When Max makes a ...|
+----------+--------------------+--------------------+
only showing top 10 rows



**Displaying data which has imdb rating 9 and 8.**

In [25]:
df1.filter(col("imdb_rating") == 9).show(10)

+-----------+--------------------+-----+----------+
|imdb_rating|               title|rated|imdb_votes|
+-----------+--------------------+-----+----------+
|        9.7|               Bluey| TV-Y|       949|
|        9.1|The Imagineering ...|  N/A|     1,194|
+-----------+--------------------+-----+----------+



In [29]:
df1.filter("imdb_rating == 8").show(10)

+-----------+--------------------+-----+----------+
|imdb_rating|               title|rated|imdb_votes|
+-----------+--------------------+-----+----------+
|        8.0|             Aladdin|    G|   356,283|
|        8.0|Beauty and the Beast|    G|   404,632|
|        8.3|    Before the Flood|   PG|    25,171|
|        8.1|     Boy Meets World| TV-G|    37,279|
|        8.3|         Brain Games| TV-G|     2,812|
|        8.4|                Coco|   PG|   333,411|
|        8.1|         Day & Night|    G|    16,108|
|        8.9|       Gravity Falls|TV-Y7|    63,771|
|        8.2|Phineas and Ferb:...| TV-G|       492|
|        8.0|Star vs. the Forc...|TV-Y7|     9,623|
+-----------+--------------------+-----+----------+
only showing top 10 rows



**Sorting**

In [32]:
df1.sort(col("title").desc()).show(10)

+-----------+--------------------+---------+--------------------+
|imdb_rating|               title|    rated|          imdb_votes|
+-----------+--------------------+---------+--------------------+
|        8.0|            Zootopia|       PG|             410,437|
|        N/A|           Zenon: Z3|     TV-G|                 N/A|
|        5.9|   Zenon: The Zequel|     TV-G|               3,318|
|        6.4|Zenon: Girl of th...|     TV-G|               6,378|
|Canada, USA|              Zapped|     TV-G|5 wins & 1 nomina...|
|        6.3|       Z-O-M-B-I-E-S|     TV-G|               2,993|
|        7.1| Your Friend the Rat|        G|               4,944|
|        6.1|           You Wish!|     TV-G|               1,608|
|        4.7|       You Lucky Dog|     TV-G|                 526|
|        6.9|    Yellowstone Cubs|Not Rated|                 177|
+-----------+--------------------+---------+--------------------+
only showing top 10 rows



**Top 10 highest-rated Disney movies based on average ratings**

In [39]:
avg_rating = df.groupBy(col("title")).agg({"imdb_votes" : "avg"}).withColumnRenamed("avg(imdb_votes)", "avg_imdb_votes")
avg_rating.orderBy(desc("avg_imdb_votes")).limit(10).show(truncate = False)


+-------------------------------------+--------------+
|title                                |avg_imdb_votes|
+-------------------------------------+--------------+
|Mr. Boogedy                          |1173.0        |
|The Incredible Dr. Pol               |999.0         |
|Elena of Avalor                      |998.0         |
|Boat Builders                        |993.0         |
|Elmer Elephant                       |991.0         |
|Lamp Life                            |980.0         |
|Roving Mars                          |968.0         |
|Bluey                                |949.0         |
|Spider-Man                           |943.0         |
|Marvel Studios: Assembling a Universe|936.0         |
+-------------------------------------+--------------+



**Disney movie which has the highest average rating**

In [49]:
avg_rating.select("title").orderBy(desc("avg_imdb_votes")).limit(1).show()

+-----------+
|      title|
+-----------+
|Mr. Boogedy|
+-----------+



**Top 10 lowest-rated Disney movies based on average ratings (not including null values)**

In [56]:
avg_rating.dropna().orderBy(asc("avg_imdb_votes")).limit(10).show(truncate = False)

+-----------------------------------------+--------------+
|title                                    |avg_imdb_votes|
+-----------------------------------------+--------------+
|Jonas Brothers: The 3D Concert Experience|1.5           |
|Out There with Jack Randall              |5.0           |
|Insider                                  |5.0           |
|The Sandlot: Heading Home                |5.0           |
|Rocky Mountain Animal Rescue             |5.0           |
|Camp Rock 2: The Final Jam               |5.2           |
|Secrets of the King Cobra                |6.0           |
|Muppet Moments                           |6.0           |
|Penguins: Life on the Edge               |6.0           |
|The Suite Life on Deck                   |6.3           |
+-----------------------------------------+--------------+



**Disney movie which has the lowest average rating**

In [58]:
avg_rating.dropna().select("title").orderBy(asc("avg_imdb_votes")).limit(1).show(truncate = False)

+-----------------------------------------+
|title                                    |
+-----------------------------------------+
|Jonas Brothers: The 3D Concert Experience|
+-----------------------------------------+



**Calculate average ratings for each genre of Disney movies**

In [188]:
genre = df.select(col("imdb_rating"), col("title"), col("rated"), col("imdb_votes"), col("genre"))
genre_avg = genre.groupBy("genre").agg(round(avg("imdb_rating"), 1)).withColumnRenamed("round(avg(imdb_rating), 1)", "avg_imdb_rating")
genre_avg.dropna().show()

+--------------------+---------------+
|               genre|avg_imdb_rating|
+--------------------+---------------+
|Animation, Advent...|            6.5|
|Animation, Comedy...|            6.7|
|Adventure, Biogra...|            8.1|
|             100 min|           84.0|
|Adventure, Comedy...|            4.7|
|Family, Romance, ...|            5.4|
|Animation, Action...|            6.4|
|Animation, Advent...|            6.1|
|Action, Adventure...|            7.0|
|Animation, Comedy...|            6.5|
|Documentary, Hist...|            7.6|
|Adventure, Family...|            8.5|
|Documentary, Biog...|            7.9|
|Documentary, Adve...|            4.7|
|  Documentary, Music|            2.3|
|Animation, Advent...|            8.4|
|Family, Comedy, A...|            7.3|
|Adventure, Comedy...|            5.3|
|Adventure, Family...|            5.9|
|Animation, Short,...|            7.6|
+--------------------+---------------+
only showing top 20 rows



**Story plot which includes ghost keyword**

In [70]:
df.select(df.columns[:3]).filter(col("plot").like("%ghost%")).show(10, truncate = False)

+---------+---------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|imdb_id  |title          |plot                                                                                                                                                            |
+---------+---------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|tt0029161|Lonesome Ghosts|Four bored ghosts in a haunted house who've scared everyone away call up Ghost Hunters Mickey, Donald, and Goofy in hopes to have a little fun scaring them off.|
+---------+---------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+



**Count the no of movie plot includes evil keyword**

In [189]:
evil_plot = df.filter(col("plot").like("%evil%"))
evil_plot.count()

25

**Grouping**

In [86]:
df.groupBy("type").count().show(truncate = False)

+-----------------------------------------------------------------------------------------------------------+-----+
|type                                                                                                       |count|
+-----------------------------------------------------------------------------------------------------------+-----+
|"" a rubber-like super-bouncy substance."                                                                  |1    |
| resident DJ for ""The Ellen Show"" ..."                                                                   |1    |
|null                                                                                                       |98   |
| Davy and Georgie                                                                                          |1    |
| an invisible creature befriends a 12 year old boy. Of course                                              |1    |
| as well as documentary footage on the lives of the three brothers."   

**Find the average IMDb rating for each director and sort the result in ascending order.**

In [190]:
avg_rating_runtime = df.groupBy("director").agg(round(avg("imdb_rating"), 1).alias("avgerage imdb rating"))\
                     .orderBy(asc("director"))
avg_rating_runtime.dropna().show(truncate = False)                   

+-------------------------------------------------------------+--------------------+
|director                                                     |avgerage imdb rating|
+-------------------------------------------------------------+--------------------+
|Aaron Blaise, Robert Walker                                  |6.8                 |
|Adam Shankman                                                |6.0                 |
|Alan Barillaro                                               |8.4                 |
|Alan Shapiro                                                 |6.5                 |
|Alan Taylor                                                  |6.9                 |
|Alastair Fothergill, Jeff Wilson                             |6.8                 |
|Alastair Fothergill, Keith Scholey, Adam Chapman(co-director)|7.4                 |
|Alastair Fothergill, Mark Linfield                           |7.2                 |
|Alex Zamm                                                    |3.

**Determine the number of movies released each year and sort them in descending order.**

In [113]:
mv_released_each_yr = df.filter(col("type") == "movie")\
                      .groupBy("year")\
                      .agg(count("imdb_id").alias("No of Movies"))\
                      .orderBy(desc("year"))
mv_released_each_yr.show()

+----+------------+
|year|No of Movies|
+----+------------+
|2020|          12|
|2019|          28|
|2018|          18|
|2017|          21|
|2016|          18|
|2015|          12|
|2014|          16|
|2013|          10|
|2012|          16|
|2011|          22|
|2010|          14|
|2009|          18|
|2008|          18|
|2007|          14|
|2006|          20|
|2005|          21|
|2004|          22|
|2003|          26|
|2002|          22|
|2001|          15|
+----+------------+
only showing top 20 rows



**Identify the top 3 directors with the most IMDb votes for their movies.**

In [147]:
changed_datatype = df.select(col("imdb_votes").cast("int"), col("director"), col("type"), col("title"))
top_five_directors = changed_datatype.filter(col("type") == "movie")\
                     .orderBy(desc("imdb_votes")).limit(3)
top_five_directors.show(truncate = False)

+----------+----------------+-----+--------------+
|imdb_votes|director        |type |title         |
+----------+----------------+-----+--------------+
|993       |Ben Sharpsteen  |movie|Boat Builders |
|991       |Wilfred Jackson |movie|Elmer Elephant|
|980       |Valerie LaPointe|movie|Lamp Life     |
+----------+----------------+-----+--------------+



**Find the average IMDb rating for movies that won awards and those that didn't.**

In [168]:
movie_with_awards = df.withColumn("has_award",when(col("awards") == "N/A", "no award").otherwise("award"))
avg_rating_by_award = movie_with_awards.groupBy("has_award")\
                     .agg(round(avg("imdb_rating"), 1).alias("average imdb rating"))
avg_rating_by_award.show()

+---------+-------------------+
|has_award|average imdb rating|
+---------+-------------------+
|    award|                7.6|
| no award|                6.7|
+---------+-------------------+



**Determine the most common language for Disney movies.**

In [175]:
most_common_lang = df.groupBy("language")\
                   .agg(count("imdb_id").alias("no of movies"))\
                   .orderBy(desc("no of movies")).limit(1)
most_common_lang.select(df.language.alias("most common language")).show()

+--------------------+
|most common language|
+--------------------+
|             English|
+--------------------+



**Calculate the average metascore and IMDb rating for movies in each genre.**

In [183]:
avg_rating_by_genre = df.groupBy("genre")\
                      .agg(round(avg("metascore"), 1).alias("average metascore"), round(avg("imdb_rating"), 1).alias("average imdb rating"))
avg_rating_by_genre.dropna().show(truncate = False)

+-----------------------------------------------------+-----------------+-------------------+
|genre                                                |average metascore|average imdb rating|
+-----------------------------------------------------+-----------------+-------------------+
|Animation, Adventure, Family, Thriller               |56.0             |6.5                |
|Adventure, Biography, Drama, Family, History         |69.0             |8.1                |
|Adventure, Comedy, Family, Fantasy, Romance          |34.0             |4.7                |
|Animation, Action, Adventure, Family, Fantasy, Sci-Fi|43.5             |6.4                |
|Action, Adventure, Fantasy                           |51.8             |7.0                |
|Animation, Comedy, Family, Fantasy, Musical          |92.0             |6.5                |
|Documentary, Biography                               |87.0             |7.9                |
|Documentary, Adventure, Biography, Family            |63.0 